In [1]:
%pip install pandas 
%pip install mysql-connector-python
%pip install decouple
%pip install pymysql sqlalchemy
%pip install python-dotenv
%pip install numpy
%pip install plotly



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

import pandas as pd
import numpy as np


from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime


import plotly.express as px
import plotly.graph_objects as go

import scipy.stats as stats
from scipy.stats import zscore


import plotly.express as px
import pandas as pd

import logging






In [3]:
logging.basicConfig(
    level=logging.INFO,  
    format='%(asctime)s - %(levelname)s - %(message)s',  # Log message format
    # filename='app.log'  # Log to a file (optional)
)

In [4]:
class DataLoader:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    def read_data(self):
        """
        handling file existence, empty dataset, and parsing errors
        """
        try:
            return pd.read_csv(self.raw_data, encoding="ISO-8859-1", engine='python')
            logging.info("read the dataset successfully")
        except FileNotFoundError:
            logging.error("Error: The file was not found.")
        except pd.errors.EmptyDataError:
            logging.error("Error: The file is empty.")
        except pd.errors.ParserError:
            logging.error("Error: The file could not be parsed.")

In [5]:
class DataCleaner:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    def percentage_of_rows_with_missing_values(self):
        try:
            rows_with_missing = self.raw_data.isnull().any(axis=1).sum()
            total_rows = len(self.raw_data)
            percentage = (rows_with_missing / total_rows) * 100
            logging.info(f"Successfully got the percentage of rows with missing values: {percentage:.2f}%")
            return percentage
        except Exception as e:
            logging.error(f"Failed to get the percentage of rows with missing values: {e}")
            return None

    def percentage_of_duplicate_rows(self):
        try:
            total_rows = len(self.raw_data)
            duplicate_rows = self.raw_data.duplicated().sum()
            duplicate_percentage = (duplicate_rows / total_rows) * 100
            logging.info(f"Successfully got the percentage of duplicate rows: {duplicate_percentage:.2f}%")
            return duplicate_percentage
        except Exception as e:
            logging.error(f"Failed to get the percentage of duplicate rows: {e}")
            return None

    def percentage_of_columns_with_missing_values(self):
        try:
            columns_with_missing = self.raw_data.isnull().any(axis=0).sum()
            total_columns = self.raw_data.shape[1]  # Corrected to use the number of columns
            percentage = (columns_with_missing / total_columns) * 100
            logging.info(f"Successfully got the percentage of columns with missing values: {percentage:.2f}%")
            return percentage
        except Exception as e:
            logging.error(f"Failed to get the percentage of columns with missing values: {e}")
            return None

    def percentage_of_duplicate_columns(self):
        try:
            total_columns = self.raw_data.shape[1]  # Corrected to use the number of columns
            duplicate_columns = self.raw_data.T.duplicated().sum()  # Transpose to check for duplicate columns
            duplicate_percentage = (duplicate_columns / total_columns) * 100
            logging.info(f"Successfully got the percentage of duplicate columns: {duplicate_percentage:.2f}%")
            return duplicate_percentage
        except Exception as e:
            logging.error(f"Failed to get the percentage of duplicate columns: {e}")
            return None

    def percentage_of_null_value_rows(self):
        try:
            rows_with_missing = self.raw_data.isnull().any(axis=1).sum()  # Count rows with any missing value
            total_rows = len(self.raw_data)
            null_percentage = (rows_with_missing / total_rows) * 100
            logging.info(f"Successfully got the percentage of rows with missing values: {null_percentage:.2f}%")
            return null_percentage
        except Exception as e:
            logging.error(f"Failed to get the percentage of null value rows: {e}")
            return None



In [6]:
if __name__ == "__main__":
    raw_dataset = '../dataset/raw/market_sales.csv'

In [7]:
    loader = DataLoader(raw_dataset)

In [8]:
    data = loader.read_data()

In [9]:
    data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Gender         1000 non-null   object 
 1   Invoice ID     1000 non-null   object 
 2   Branch         1000 non-null   object 
 3   City           1000 non-null   object 
 4   Customer type  1000 non-null   object 
 5   Product line   1000 non-null   object 
 6   Unit price     1000 non-null   float64
 7   Quantity       1000 non-null   int64  
 8   Tax 5%         1000 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 70.4+ KB


In [10]:
    data.columns

Index(['Gender', 'Invoice ID', 'Branch', 'City', 'Customer type',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%'],
      dtype='object')

In [11]:
    print(data)

     Gender   Invoice ID Branch       City Customer type  \
0    Female  750-67-8428      A     Yangon        Member   
1    Female  226-31-3081      C  Naypyitaw        Normal   
2    Female  355-53-5943      A     Yangon        Member   
3    Female  315-22-5665      C  Naypyitaw        Normal   
4    Female  665-32-9167      A     Yangon        Member   
..      ...          ...    ...        ...           ...   
995    Male  745-74-0715      A     Yangon        Normal   
996    Male  690-01-6631      B   Mandalay        Normal   
997    Male  233-67-5758      C  Naypyitaw        Normal   
998    Male  727-02-1313      A     Yangon        Member   
999    Male  347-56-2442      A     Yangon        Normal   

               Product line  Unit price  Quantity    Tax 5%  
0         Health and beauty       74.69         7  261.4150  
1    Electronic accessories       15.28         5    3.8200  
2    Electronic accessories       68.84         6   20.6520  
3        Home and lifestyle    

In [12]:
    cleaner = DataCleaner(data)

# Percentage of rows with missing values
There are no rows with missing values

In [14]:
    cleaner.percentage_of_rows_with_missing_values()

2025-03-17 00:12:14,171 - INFO - Successfully got the percentage of rows with missing values: 0.00%


0.0

# Percentage of duplicate rows
There are no duplicate rows

In [16]:
    cleaner.percentage_of_duplicate_rows()

2025-03-17 00:12:14,193 - INFO - Successfully got the percentage of duplicate rows: 0.00%


0.0

# Percentage of columns with missing values

In [18]:
    cleaner.percentage_of_columns_with_missing_values()

2025-03-17 00:12:14,216 - INFO - Successfully got the percentage of columns with missing values: 0.00%


0.0

# Percentage of duplicate columns
There are no duplicate columns

In [20]:
    cleaner.percentage_of_duplicate_columns()

2025-03-17 00:12:14,264 - INFO - Successfully got the percentage of duplicate columns: 0.00%


0.0

# Percentage of rows with null values

In [21]:
    cleaner.percentage_of_null_value_rows()

2025-03-17 00:12:14,269 - INFO - Successfully got the percentage of rows with missing values: 0.00%


0.0

In [45]:
    data.columns

Index(['Gender', 'Invoice ID', 'Branch', 'City', 'Customer type',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%'],
      dtype='object')

# Gender attributes are correct

In [47]:
    data['Gender'].unique()

array(['Female', 'Male'], dtype=object)

# The Branches
- A
- B
- C

In [51]:
    data['Branch'].unique()

array(['A', 'C', 'B'], dtype=object)

# The Cities
- Yangon
- Naypyitaw
- Mandalay

In [55]:
    data['City'].unique()

array(['Yangon', 'Naypyitaw', 'Mandalay'], dtype=object)

# Customer types
- Normal
- Member

In [59]:
    data['Customer type'].unique()

array(['Member', 'Normal'], dtype=object)

# Product line
- Health and beauty
- Electronic accessories
- Home and lifestyle
- Food and beverages
- Fashion accessories
- Sports and travel

In [62]:
    data['Product line'].unique()

array(['Health and beauty', 'Electronic accessories',
       'Home and lifestyle', 'Food and beverages', 'Fashion accessories',
       'Sports and travel'], dtype=object)

# The prices are from 10.08 to 99.96

In [103]:
    print(f"{data['Unit price'].min()}-{data['Unit price'].max()}")

10.08-99.96


# The qunatities range between 1-10

In [99]:
    print(f"{data['Quantity'].min()}-{data['Quantity'].max()}")

1-10
